In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

blast_result = "../raw_data/blast_res.txt"
mmseqs_result = "../mmseqs_toxins/results.tsv"
headers = ["query","target","fident","alnlen","mismatch","gapopen","qstart","qend","tstart","tend","evalue","bits"]
# headers = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart", "send", "evalue", "bitscore"]

In [2]:
# --- read BLAST results ---
# df = pd.read_csv(blast_result, sep="\t", header=None, names=headers)
# df["fident"] = df["fident"] / 100

# --- read mmseqs results ---
df = pd.read_csv(mmseqs_result, sep="\t")
df

,query,target,fident,evalue,bits
0,S0F1M9,S0F1M9,1.000,5.672000e-49,162
1,S0F1M9,S0F1P1,0.939,2.678000e-44,148
2,S0F1M9,P0DMQ4,0.824,9.828000e-37,126
3,S0F1M9,P82228,1.000,1.009000e-21,83
4,S0F1M9,S0F1M6,0.532,6.265000e-20,78
...,...,...,...,...,...
26842717,P0C2P4,P84745,1.000,1.439000e+05,7
26842718,P0C2P4,P0CJ34,1.000,1.439000e+05,7
26842719,P0C2P4,P0DM54,1.000,1.439000e+05,7
26842720,P0C2P4,P0DKZ9,0.610,1.966000e+05,7


In [3]:
def process_similarity_values(
    values: pd.Series, metric: str
) -> pd.Series:
    """Processes similarity values based on their meaning."""
    if metric == "evalue":
         # Convert evalues to similarities: -log10(evalue)
        processed = -np.log10(values.replace(0, 1e-300))
    elif metric == "bits":
        # Higher bits scores indicate better alignment
        processed = values
    elif metric == "rmsd":
        # Invert and scale RMSD: smaller RMSD = higher similarity
        max_rmsd = values.max()
        processed = (max_rmsd - values) / max_rmsd
        print(f"RMSD range after processing: {processed.min()} to {processed.max()}")
    else:
        processed = values

    if processed.min() != processed.max():
        processed = (processed - processed.min()) / (
            processed.max() - processed.min()
        )
    return processed.round(3)


In [4]:
# Get unique sequence IDs and create mapping to integers
all_seqs = sorted(list(set(df['query'].unique()) | set(df['target'].unique())))
seq_to_idx = {seq: idx for idx, seq in enumerate(all_seqs)}

# Convert sequences to integer indices
row_idx = np.array([seq_to_idx[seq] for seq in df['query']])
col_idx = np.array([seq_to_idx[seq] for seq in df['target']])

# Create sparse matrix in COO format
n = len(all_seqs)

col = "evalue"
processed_values = process_similarity_values(df[col], col)

sparse_matrix = coo_matrix(
    (df[col], (row_idx, col_idx)),
    shape=(n, n)
)

# Convert to dense matrix
dense_matrix = sparse_matrix.toarray()

# Convert to DataFrame with proper labels
matrix = pd.DataFrame(
    dense_matrix,
    index=all_seqs,
    columns=all_seqs
)

# replace NaN with 0s
# matrix = matrix.replace(0, np.nan)

# matrix = matrix.round(3)
# matrix = 1 - matrix

In [5]:
count_zeros = np.count_nonzero(matrix == 0)
count_zeros

468

In [6]:
output_file = "../processed_data/evalue.csv"
matrix.to_csv(output_file, sep=',')

In [23]:
import numpy as np
from sklearn.manifold import MDS
import networkx as nx
from scipy.sparse.csgraph import shortest_path

import numpy as np
from sklearn.manifold import MDS
import networkx as nx
from scipy.sparse.csgraph import shortest_path
from scipy.sparse import csr_matrix

def handle_sparse_distance_matrix(dist_matrix):
    """
    Handle sparse distance matrix with NaN values using multiple approaches.

    Parameters:
    dist_matrix: numpy array
        Distance matrix with NaN values

    Returns:
    dict: Dictionary containing different MDS embeddings
    """
    # Make a copy and ensure float dtype
    D = dist_matrix.astype(np.float64)

    # Approach 1: Fill with mean of non-NaN values
    mean_dist = np.nanmean(D)
    D_mean = D.copy()
    np.fill_diagonal(D_mean, 0)  # Ensure diagonal is 0
    D_mean[np.isnan(D_mean)] = mean_dist

    # Approach 2: Use shortest path distances
    # Create sparse matrix directly instead of using NetworkX
    n = D.shape[0]
    rows, cols, weights = [], [], []

    for i in range(n):
        for j in range(i+1, n):
            if not np.isnan(D[i,j]):
                rows.extend([i, j])
                cols.extend([j, i])
                weights.extend([D[i,j], D[i,j]])

    # Create sparse matrix directly
    sparse_matrix = csr_matrix((weights, (rows, cols)), shape=(n, n))

    # Compute shortest paths
    D_path = shortest_path(sparse_matrix)

    # Handle any infinities from disconnected components
    D_path[~np.isfinite(D_path)] = mean_dist * 2  # Use twice the mean as an estimate

    # Perform MDS with different approaches
    results = {}

    # Standard MDS with mean-filled distances
    mds_mean = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
    results['mean_filled'] = mds_mean.fit_transform(D_mean)

    # MDS with shortest path distances
    mds_path = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
    results['shortest_path'] = mds_path.fit_transform(D_path)

    return results


# Apply MDS
embeddings = handle_sparse_distance_matrix(matrix.to_numpy())

# Access results
mean_filled_coords = embeddings['mean_filled']
shortest_path_coords = embeddings['shortest_path']

NegativeCycleError: Negative cycle detected on node 0